In [23]:
import torch as nn
import numpy as np
import pandas as pd

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

class SingleScoreCNN(nn.Module):
    def __init__(self):
        super(SingleScoreCNN, self).__init__()
        # Define the layers of the CNN
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 64 * 8)  # Adjust the size based on the input size
        self.fc2 = nn.Linear(64 * 8, 64 * 2)  
        self.fc3 = nn.Linear(64 * 2, 64) 
        self.fc4 = nn.Linear(64, 1)
    
    def forward(self, x):
        # Define the forward pass
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)
        return x

# Example usage:
model = SingleScoreCNN()
output = model(torch.randn(1, 3, 64, 64))  # Example input tensor (batch_size, channels, height, width)

# Define the loss and optimizer
criterion = nn.MSELoss()  # For example, if it's a regression task
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [25]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class MeasuresDataset(Dataset):
    def __init__(self, csv_file, image_dir, transform=None, indices=None):
        data = pd.read_csv(csv_file)
        if indices is not None:
            self.data = data.iloc[indices]
        else:
            self.data = data
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        page = str(int(self.data.iloc[idx]['Page']))
        label = self.data.iloc[idx]['Final Score']

        img_path = os.path.join(self.image_dir, f"{page}.jpg")
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float), img_path

# Define transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to the size expected by the network
    transforms.ToTensor()
])

# Load dataset
dataset = MeasuresDataset(csv_file='measures_context.csv', image_dir='./combined', transform=transform)

# Splitting data indices for train and test
total_size = len(dataset)
testsizepercent = 0.2
test_size = int(testsizepercent * total_size)
train_size = total_size - test_size

indices = torch.randperm(total_size).tolist()
train_indices = indices[test_size:]
test_indices = indices[:test_size]

train_dataset = MeasuresDataset(csv_file='measures_context.csv', image_dir='./combined', transform=transform, indices=train_indices)
test_dataset = MeasuresDataset(csv_file='measures_context.csv', image_dir='./combined', transform=transform, indices=test_indices)

# Create data loaders
dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



In [27]:
import torch
import torch.optim as optim
import torch.nn as nn

# Prepare model, loss function, and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 20
device = "mps"

model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    for images, labels, img_paths in dataloader:
        # Move data to GPU if available
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()  # Zero the parameter gradients
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs.squeeze(), labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Accumulate the loss
        epoch_loss += loss.item()
    
    # Compute and print the average loss for the epoch
    avg_loss = epoch_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], MSE Loss: {avg_loss:.4f}")

Epoch [1/20], MSE Loss: 0.0435
Epoch [2/20], MSE Loss: 0.0100
Epoch [3/20], MSE Loss: 0.0099
Epoch [4/20], MSE Loss: 0.0096
Epoch [5/20], MSE Loss: 0.0096


KeyboardInterrupt: 

In [28]:
torch.save(model,"simple_model.pth")

In [29]:
# model = torch.load("simple_model.pth")

In [30]:
model

SingleScoreCNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=64, bias=True)
  (fc4): Linear(in_features=64, out_features=1, bias=True)
)

In [31]:
thresholds = [0.3, 0.4, 0.6, 0.8, 1]
def model_predict_class(image, model):
    scores = model(image)
    scores = scores.squeeze()
    print(scores)
    categories = []
    for score in scores:
        print(score)
        if score <= thresholds[0]:
            category = "Very Poor"
        elif thresholds[0] < score <= thresholds[1]:
            category = "Poor"
        elif thresholds[1] < score <= thresholds[2]:
            category = "Fair"
        elif thresholds[2] < score <= thresholds[3]:
            category = "Good"
        else:
            category = "Very Good"
        categories.append(category)    
    return categories     

In [32]:
device = "mps"
model = model.to(device)

In [33]:
for images, labels, img_paths in dataloader:
    images, labels = images.to(device), labels.to(device)
    output = model_predict_class(images, model)
    for img_path, o, l in zip(img_paths, output, labels):
        print(f"{img_path}: {o} (Label: {l.item()})")

tensor([0.4145, 0.4246, 0.4650, 0.4014, 0.3983, 0.4805, 0.5285, 0.4151, 0.4359,
        0.4320, 0.4561, 0.4285, 0.4060, 0.4282, 0.4070, 0.4338, 0.4281, 0.4846,
        0.4397, 0.4398, 0.4266, 0.3973, 0.4141, 0.3987, 0.4103, 0.3947, 0.4087,
        0.4149, 0.4765, 0.4108, 0.4138, 0.3971], device='mps:0',
       grad_fn=<SqueezeBackward0>)
tensor(0.4145, device='mps:0', grad_fn=<UnbindBackward0>)
tensor(0.4246, device='mps:0', grad_fn=<UnbindBackward0>)
tensor(0.4650, device='mps:0', grad_fn=<UnbindBackward0>)
tensor(0.4014, device='mps:0', grad_fn=<UnbindBackward0>)
tensor(0.3983, device='mps:0', grad_fn=<UnbindBackward0>)
tensor(0.4805, device='mps:0', grad_fn=<UnbindBackward0>)
tensor(0.5285, device='mps:0', grad_fn=<UnbindBackward0>)
tensor(0.4151, device='mps:0', grad_fn=<UnbindBackward0>)
tensor(0.4359, device='mps:0', grad_fn=<UnbindBackward0>)
tensor(0.4320, device='mps:0', grad_fn=<UnbindBackward0>)
tensor(0.4561, device='mps:0', grad_fn=<UnbindBackward0>)
tensor(0.4285, device=